In [ ]:
%matplotlib inline
DEPOT_THUMBNAIL_PATH="/scratch/tb/thumbnails/"
DCM_PATH="/scratch/tb/cxr_hq/"
TEST_PATH="/scratch/tb/cxr/"
NUMBER_SAMPLES=500
TEST_NUMBER_SAMPLES=NUMBER_SAMPLES
RANDOM_SEED=13
SAMPLE_IMAGE_SIZE=64

In [ ]:
from glob import glob
import random
import functools
import multiprocessing

from math import sqrt

import SimpleITK as sitk

import pandas

import numpy as np
from numpy import linalg as LA

from sklearn import decomposition
from sklearn.covariance import EmpiricalCovariance, MinCovDet
from sklearn.naive_bayes import GaussianNB
from sklearn.covariance import EllipticEnvelope

import matplotlib.pyplot as plt

from myshow import myshow

import pickle

In [ ]:
from tbpcxr.utilities import read_dcm, normalize_img
import tbpcxr.registration 
from tbpcxr.model import PCAModel

In [ ]:
from importlib import reload  
reload(tbpcxr.model)
reload(tbpcxr.registration)
PCAModel = tbpcxr.model.PCAModel


Create two list of image files a *traning* set and a *validation* set. The training set is used to build the PCAModel and the validation set is used to verify that the model is not over fitted at several stages.



In [ ]:

test_file_list = glob(TEST_PATH+"/**.dcm")

print( "Found {0} DICOM.".format(len(test_file_list)))
test_file_list = random.sample(test_file_list, TEST_NUMBER_SAMPLES)

In [ ]:

# Seach a directory for files to use
#file_list = glob(DEPOT_THUMBNAIL_PATH+"/**/*thumbnail.png", recursive=True)
#print( "Found {0} thumbnails.".format(len(file_list)))

# Currently, from one large set of high quality CXR data we are randomly choosing part
# for the training and another part for validation.


dcm_list = glob(DCM_PATH+"/**.dcm")

print( "Found {0} DICOM.".format(len(dcm_list)))


random.seed(RANDOM_SEED)

if NUMBER_SAMPLES*2 > len(dcm_list):
    print("Need to reuse samples for train and validation")
    train_file_list = random.sample(dcm_list, NUMBER_SAMPLES)
    validation_file_list = random.sample(dcm_list, NUMBER_SAMPLES)
else:
    fl = random.sample(dcm_list, NUMBER_SAMPLES*2)
    train_file_list = fl[:NUMBER_SAMPLES]
    validation_file_list = fl[NUMBER_SAMPLES:]


In [ ]:


def read_and_normalize(f):
        try:
            img = read_dcm(f)
        except:
            img = sitk.ReadImage(f, sitk.sitkFloat32)
            
        return normalize_img(img)

    
def tile_with_scale(image_list, width=10):
    img = sitk.Tile([ sitk.RescaleIntensity(img, outputMinimum=0, outputMaximum=255) for img in image_list], [width,0])
    return sitk.Cast(img, sitk.sitkUInt8)


Use multi-processing to read and regularize the data with multiple cores.

In [ ]:

with multiprocessing.Pool() as pool:
    train_set = pool.map(read_and_normalize, train_file_list)
    validation_set = pool.map(read_and_normalize, validation_file_list)
    
test_set = list(map(read_and_normalize, test_file_list))

In [ ]:
print(len(train_set))
tile_width=min(int(sqrt(NUMBER_SAMPLES)), 25)
myshow(tile_with_scale(train_set, width=tile_width), title="Training Set")
myshow(tile_with_scale(validation_set, width=tile_width), title="Validation Set")

In [ ]:
atlas_size_factor=2
atlas = tbpcxr.registration.build_atlas(sitk.Expand(pca.image_ref, [atlas_size_factor]*2), train_set)

In [ ]:
sitk.WriteImage(atlas, "cxr_atlas.nrrd")

In [ ]:
pca = PCAModel()
pca.image_atlas = sitk.ReadImage("cxr_atlas.nrrd")
myshow(pca.image_atlas, title="CXR Atlas")
crop_size = crop_size = [int(s * pca.CROP_SIZE/pca.SAMPLE_IMAGE_SIZE) for s in pca.image_atlas.GetSize()]
myshow(sitk.Crop(pca.image_atlas, crop_size, crop_size))

In [ ]:
train_reg_set = [ pca.register_to_atlas_and_resample(img) for img in train_set ]

In [ ]:
img = sitk.Tile( train_reg_set,  [tile_width,0])
imgc = sitk.Tile( [sitk.BinShrink(sitk.Cast(atlas, sitk.sitkFloat32), [atlas_size_factor]*2)]*len(train_reg_set), [tile_width,0])

myshow( sitk.Cast(sitk.RescaleIntensity(sitk.Compose(img, img+imgc, img)), sitk.sitkVectorUInt8))

In [ ]:

reg = pca.register_to_atlas_and_resample(train_set[2], verbose=2)

myshow(pca.image_atlas, title="CXR Atlas")

myshow(reg, title="Registered")
myshow(sitk.Cast(sitk.RescaleIntensity(sitk.Compose(reg, sitk.Resample(pca.image_atlas, reg, outputPixelType=sitk.sitkFloat32), reg)), sitk.sitkVectorUInt8))

In [ ]:
validation_reg_set = [ pca.register_to_atlas_and_resample(img) for img in validation_set]
myshow(tile_with_scale(validation_reg_set, width=tile_width))
sitk.Show(tile_with_scale(validation_reg_set, width=tile_width))

In [ ]:
test_reg_set = [ pca.register_to_atlas_and_resample(img) for img in test_set ]
myshow(tile_with_scale(test_reg_set, width=tile_width))
sitk.Show(tile_with_scale(test_reg_set, width=tile_width))

In [ ]:
# Convert the list of images into a numpy array cropped, for the PCA space
train_vec = pca._images_to_arr(train_reg_set)
validation_vec = pca._images_to_arr(validation_reg_set)
test_vec = pca._images_to_arr(test_reg_set)

In [ ]:
# The the PCA computation for a variety of number of components

# Compare the residuals of the training data-set to the validation to avoid over fitting the training data
mean_res = []
min_res = []
max_res = []
x_res = []
validation_res = []

for n_component in range(1,min(100,train_vec.shape[0]-1),2):
    
    pca.compute(train_vec, n_component)
    
    residuals = pca.residuals(train_vec)
    
    mean_res.append(np.mean(residuals))
    min_res.append(np.min(residuals))
    max_res.append(np.max(residuals))
    x_res.append(n_component)
    
    residuals = pca.residuals(validation_vec)
    validation_res.append(np.mean(residuals))

In [ ]:
fig, ax = plt.subplots()
ax.fill_between(x_res, max_res, min_res ,  alpha=0.2, label="Training Min/Max")
ax.plot(x_res, mean_res, label="Training Mean")
ax.plot(x_res, validation_res, label="Validation Mean")
ax.set_title("PCA Image residuals")
ax.set_xlabel("number of components")
ax.set_ylabel("RMS residual")
ax.legend(loc='upper right')

In [ ]:
number_of_components=25
outlier_dev=6

pca = PCAModel()
pca.image_atlas = atlas
pca.compute(train_vec, 100)


# Re-compute PCA dropping the outliers base on Mahalanobis

rds = pca.robust_distance(train_vec)
rds_threshhold = np.quantile(rds, 0.95)
idxs = np.where(rds < rds_threshhold)[0]
outlier_idxs = np.where(rds >= rds_threshhold)[0]

plt.hist(rds, 128)
plt.title("Full Training - Mahalanobis")
plt.show()


res = pca.residuals(train_vec)
print("Residuals\n\tMin: {0}\n\tMean: {1}\n\tMedian: {2}\n\tMax: {3} ".format( np.min(res), np.mean(res), np.median(res), np.max(res)))

plt.hist(res, 128)
plt.title("Full Training -  residuals")
plt.show()

print("Number of outliers {0}".format(len(outlier_idxs)))
myshow(tile_with_scale([train_reg_set[i] for i in outlier_idxs], 5), title="Rejected Registered")
sitk.Show(tile_with_scale([train_set[i] for i in outlier_idxs], 5), title="Rejected Original")


train2_vec = train_vec[idxs]


pca.compute(train2_vec, number_of_components)

res = pca.residuals(train_vec)

print("Residuals\n\tMin: {0}\n\tMean: {1}\n\tMedian: {2}\n\tMax: {3} ".format( np.min(res), np.mean(res), np.median(res), np.max(res)))


rds2 = pca.robust_distance(train_vec)
outlier_idxs = np.where(rds2 >= rds_threshhold)[0]
print("Number of outliers {0}".format(len(outlier_idxs)))




plt.hist(res, 128)
plt.title("With Outliers Removed")
plt.show()

plt.hist(pca.residuals(validation_vec), 128)
plt.title("Test Images")
plt.show()



In [ ]:
pca.image_atlas = atlas
pkl_filename = "cxr_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(pca, file)

In [ ]:
max_n = 20
res = pca.residuals(train_vec)
idxs = np.argsort(res)[-1:-(max_n+1):-1]

print("res: {}".format(res[:max_n]))

print("max res: {}".format(res[idxs]))


print("max res: {}".format(idxs))
    
sitk.Show(tile_with_scale(train_reg_set[:max_n], 5), title="First Images")
sitk.Show(tile_with_scale(pca.restored_images(train_vec[:max_n,:])), title="First Image Reconstruction")

sitk.Show(tile_with_scale([train_reg_set[i] for i in idxs], 5) , title="Image with Max PCA residuals")
sitk.Show(tile_with_scale([train_set[i] for i in idxs], 5) , title="Image with Max PCA residuals")
myshow(tile_with_scale(pca.restored_images(train_vec[idxs,:]), 5), title="Image Reconstruction with Max PCA residuals")



In [ ]:
comps = pca._arr_to_images(pca.pca.components_)
img = tile_with_scale(comps, 5)

sitk.WriteImage(img, "components.png")
myshow(img)

In [ ]:
myshow(tile_with_scale(test_reg_set, width=tile_width), title="Training Set")

In [ ]:

train_res = pca.residuals(train_vec)
train_dist = pca.robust_distance(train_vec)


validation_res = pca.residuals(validation_vec)
validation_dist = pca.robust_distance(validation_vec)


test_res = pca.residuals(test_vec)
test_dist = pca.robust_distance(test_vec)



plt.plot(train_res, train_dist, ls='none', marker='.', label="training", alpha=0.5, markersize=2)

plt.plot(validation_res, validation_dist, ls='none', marker='.', label="validation", alpha=0.5, markersize=2)
plt.plot(test_res, test_dist, ls='none', marker='.', label="testing", alpha=0.5, markersize=2)
plt.xlabel("PCA Residuals")
plt.ylabel("PCA Mahalanobis Distance")



X = np.stack((train_res, train_dist), axis=-1)
train_cov =  MinCovDet().fit(X)


X = np.stack((validation_res, validation_dist), axis=-1)
validation_cov =  MinCovDet().fit(X)

X = np.stack((test_res, test_dist), axis=-1)
test_cov =  MinCovDet().fit(X)

xx, yy = np.meshgrid(np.linspace(plt.xlim()[0], plt.xlim()[1], 100),
                     np.linspace(plt.ylim()[0], plt.ylim()[1], 100))
zz = np.c_[xx.ravel(), yy.ravel()]


mahal_emp_cov = train_cov.mahalanobis(zz)
mahal_emp_cov = mahal_emp_cov.reshape(xx.shape)
emp_cov_contour = plt.contour(xx, yy, np.sqrt(mahal_emp_cov))


mahal_emp_cov = validation_cov.mahalanobis(zz)
mahal_emp_cov = mahal_emp_cov.reshape(xx.shape)
emp_cov_contour = plt.contour(xx, yy, np.sqrt(mahal_emp_cov),
                                  linestyles='dashed')


mahal_emp_cov = test_cov.mahalanobis(zz)
mahal_emp_cov = mahal_emp_cov.reshape(xx.shape)
emp_cov_contour = plt.contour(xx, yy, np.sqrt(mahal_emp_cov),
                                  linestyles='dotted')

plt.legend()

In [ ]:
fig, axs = plt.subplots(2, 2)


train_mahalanobis = train_dist

axs[0,0].set_title("Traning PCA Residuals")
axs[0,0].hist(train_res, 128)
axs[0,1].set_title("Traning PCA Mahalanobis")
axs[0,1].hist(train_mahalanobis, 128)


test_mahalanobis = test_dist

axs[1,0].set_title("Testing PCA Residuals")
axs[1,0].hist(test_res, 128)
axs[1,1].set_title("Testing PCA Mahalanobis")
axs[1,1].hist(test_mahalanobis, 128)

# the shared labels needs work for this chart
# Hide x labels and tick labels for top plots and y ticks for right plots.
#for ax in axs.flat:
#    ax.label_outer()

print("Training PCA Residuals Quantiles: \n\t0.50: {0}\n\t0.95: {1}\n\t0.99: {2}".
     format(np.quantile(train_res, 0.50),
            np.quantile(train_res, 0.95),
            np.quantile(train_res, 0.99)))
print("Training Mahalanobis Quantiles: \n\t0.50: {0}\n\t0.95: {1}\n\t0.99: {2}".
     format(np.quantile(train_mahalanobis, 0.50),
            np.quantile(train_mahalanobis, 0.95),
            np.quantile(train_mahalanobis, 0.99)))

threshold_res= np.quantile(train_res, 0.98)
threshold_mahalanobis = np.quantile(train_mahalanobis, 0.98)




In [ ]:
X = np.stack((train_res, train_dist), axis=-1)

# The construction of the EllipticEnvelope has been moved to the PCAModel.compute method
outlier_detector = EllipticEnvelope(contamination=0.10)
outlier_detector.fit(X)

#X = np.stack((test_res, test_dist), axis=-1)
y_pred = outlier_detector.predict(X)



# Compare given classifiers under given settings
xx, yy = np.meshgrid(np.linspace(0, np.max(X[:,0]), 150),
                     np.linspace(0, np.max(X[:,1]), 150))

Z = outlier_detector.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='black')

colors = np.array(['#377eb8', '#ff7f00'])
plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[(y_pred + 1) // 2])
plt.xlabel("PCA Residuals")
plt.ylabel("PCA Mahalanobis Distance")


print(np.count_nonzero(y_pred<0))


outliers = np.argwhere(y_pred < 0).flatten()
myshow(tile_with_scale([train_set[idx] for idx in outliers], width=5))

In [ ]:

X = np.stack((test_res, test_dist), axis=-1)
y_pred = outlier_detector.predict(X)



# Compare given classifiers under given settings
xx, yy = np.meshgrid(np.linspace(0, np.max(X[:,0]), 150),
                     np.linspace(0, np.max(X[:,1]), 150))

Z = outlier_detector.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='black')

colors = np.array(['#377eb8', '#ff7f00'])
plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[(y_pred + 1) // 2])


outliers = np.argwhere(y_pred < 0).flatten()
myshow(tile_with_scale([test_set[idx] for idx in outliers], width=5))
sitk.Show(tile_with_scale([test_set[idx] for idx in outliers], width=5))

X = np.array( [[test_res[0], test_dist[1]]])
y_pred = outlier_detector.predict(X)